In [ ]:
import datetime
centralDataset=[]
# date=datetime.datetime.today()
date=datetime.date(2017, 7, 1)
sec="https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAH001_G_CA_D.001%2F2023%2F01%2FFLDAS_NOAH001_G_CA_D.A20230101.001.nc&SHORTNAME=FLDAS_NOAH001_G_CA_D&VARIABLES=Evap_tavg&SERVICE=L34RS_LDAS&FORMAT=dGlmLw&DATASET_VERSION=001&LABEL=FLDAS_NOAH001_G_CA_D.A20230101.001.nc.SUB.tif&BBOX=24.562%2C86.152%2C25.153%2C86.612&VERSION=1.02%27"
ET = []
centralnames=[]
for i in range(365):
    year=str(date.year)
    month=str(date.month).zfill(2)
    day=str(date.day).zfill(2)
    fullDate = date.strftime('%Y%m%d')
    filename = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FFLDAS%2FFLDAS_NOAH001_G_CA_D.001%2F"+year+"%2F"+month+"%2FFLDAS_NOAH001_G_CA_D.A"+fullDate+".001.nc&SERVICE=L34RS_LDAS&BBOX=21%2C65.566%2C37.932%2C99.844&FORMAT=Y29nLw&VERSION=1.02&SHORTNAME=FLDAS_NOAH001_G_CA_D&LABEL=FLDAS_NOAH001_G_CA_D.A"+fullDate+".001.nc.SUB.tif&DATASET_VERSION=001&VARIABLES=Evap_tavg"
    print(filename)
    outputName = fullDate+".tif"
    print(date.strftime('%Y%m%d'), year, month, day, filename)
    date += datetime.timedelta(days=1)
    centralnames.append(outputName)

In [ ]:
!pip install rasterio
import rasterio as rio
import numpy as np
from PIL import Image
import rasterio
import copy

In [ ]:
len(centralnames)

365

In [ ]:
res=0

index=0
with rasterio.open('/content/'+centralnames[0]) as src1:
  image1 = src1.read()
  profile1 = src1.profile

for i in range(1, 365):
    try:
      with rasterio.open('/content/'+centralnames[i]) as src2:
        image2 = src2.read()
        profile2 = src2.profile
    except:
      continue
    print(centralnames[i])
    if image1.shape != image2.shape:
      raise ValueError("Images must have the same shape")
    # Add the images together
    result = image1 + image2
    image1 = copy.deepcopy(result)
    # Update the profile with the result's shape
profile1.update(count=result.shape[0])


# Write the resulting image with the preserved CRS
with rasterio.open('result_2017_2018_Hydroyear.tif', 'w', **profile1) as dst:
    dst.write(result)


with rasterio.open('/content/20170701.tif') as src:
    with rasterio.open('result_2017_2018_Hydroyear.tif', 'r+', **src.profile) as dst:
        dst.write(dst.read())

